In [5]:
import os
import json
from openai import OpenAI

In [6]:
client = OpenAI(api_key="key")

TELEGRAM_TOKEN = "8539825563:AAHuHzqk3JJpV24ZXSfOoIP7ANefjsTqOwM"

In [9]:

def test_model(prompt: str) -> dict:
    response = client.chat.completions.create(
        model="gpt-4.1-mini",  # замените на доступную вам модель
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    content = response.choices[0].message.content
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON", "raw": content}

if __name__ == "__main__":
    prompt = "Извлеките поля: номер документа, дата, общая сумма. Верните только JSON."
    output = test_model(prompt)
    print(json.dumps(output, indent=2, ensure_ascii=False))


{
  "error": "Invalid JSON",
  "raw": "```json\n{\n  \"номер документа\": null,\n  \"дата\": null,\n  \"общая сумма\": null\n}\n```"
}


In [13]:
import os
import json
import base64
from openai import OpenAI



MODEL_NAME = "gpt-4.1-mini"  # замените на доступную модель

def extract_from_file(file_path: str) -> dict:
    # читаем файл
    with open(file_path, "rb") as f:
        file_bytes = f.read()
    b64 = base64.b64encode(file_bytes).decode("utf-8")

    # определяем MIME-тип на основе расширения
    _, ext = os.path.splitext(file_path.lower())
    if ext == ".pdf":
        mime = "application/pdf"
    elif ext in [".jpg", ".jpeg"]:
        mime = "image/jpeg"
    elif ext == ".png":
        mime = "image/png"
    else:
        mime = "application/octet-stream"

    # Формируем input для мультимодальной модели
    response = client.responses.create(
        model=MODEL_NAME,
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_file", "filename": os.path.basename(file_path), "file_data": f"data:{mime};base64,{b64}"},
                    {"type": "input_text", "text": """


                    Вы — ассистент по извлечению данных из бухгалтерских документов.
Просмотрите весь документ и извлеките следующие данные:
1. Номер документа
2. Дата документа (формат «ДД.MM.ГГГГ»)
3. Реквизиты продавца:
   • название
   • ИНН
   • ОГРН (если есть)
   • адрес
   • телефон (если есть)
   • e-mail (если есть)
4. Реквизиты покупателя:
   • название
   • ИНН
   • ОГРН (если есть)
   • адрес
   • телефон (если есть)
   • e-mail (если есть)
5. Все номенклатуры (товары или услуги) с каждой строкой:
   • название позиции
   • количество
   • единица измерения
   • цена за единицу
   • сумма по позиции
6. Общая сумма документа (в рублях, с двумя знаками после запятой).

Требования к выводу:
• Верните **только** JSON-объект со следующей структурой:
```json
{
  "document_number": "...",
  "document_date": "...",
  "seller": {
    "name": "...",
    "inn": "...",
    "ogrn": "...",
    "address": "...",
    "phone": "...",
    "email": "..."
  },
  "buyer": {
    "name": "...",
    "inn": "...",
    "ogrn": "...",
    "address": "...",
    "phone": "...",
    "email": "..."
  },
  "items": [
    {
      "name": "...",
      "quantity": "...",
      "unit": "...",
      "unit_price": "...",
      "line_total": "..."
    }
    // … (повтор для каждой позиции)
  ],
  "total_amount": "..."
}

"""}
                ]
            }
        ],
        temperature=0
    )

    content = response.output_text if hasattr(response, "output_text") else response.choices[0].message.content
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON", "raw": content}

if __name__ == "__main__":
    # Укажите путь к вашему файлу
    file_path = input("Введите путь к файлу (PDF / JPG / PNG): ").strip()
    if not os.path.isfile(file_path):
        print("Файл не найден:", file_path)
    else:
        result = extract_from_file(file_path)
        print(json.dumps(result, indent=2, ensure_ascii=False))




Введите путь к файлу (PDF / JPG / PNG): /content/PDF000.pdf
{
  "error": "Invalid JSON",
  "raw": "```json\n{\n  \"document_number\": \"471\",\n  \"document_date\": \"30.05.2025\",\n  \"seller\": {\n    \"name\": \"ООО \\\"ТД КОНТУР\\\"\",\n    \"inn\": \"7726407260\",\n    \"ogrn\": \"1177746001001\",\n    \"address\": \"117545, г. Москва, вн.тер. муниципальный округ Чертаново Северное, ул. Днепропетровская, д. 45, пом. IV/19\",\n    \"phone\": \"\",\n    \"email\": \"\"\n  },\n  \"buyer\": {\n    \"name\": \"ООО \\\"МАКСИМА ГРУПП\\\"\",\n    \"inn\": \"5029148428\",\n    \"ogrn\": \"\",\n    \"address\": \"190013, г. Санкт-Петербург, вн.тер. муниципальный округ Семеновский, пр-кт Московский, д. 18, пом. 1Н\",\n    \"phone\": \"\",\n    \"email\": \"\"\n  },\n  \"items\": [\n    {\n      \"name\": \"Профиль С-образный усиленный (ПС-63-55-239) с полимерным покрытием\",\n      \"quantity\": \"442,43\",\n      \"unit\": \"пог. м\",\n      \"unit_price\": \"43,186\",\n      \"line_total\"

# ТГ Бот

In [32]:
!pip install aiogram

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.2/698.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.13.1
    Uninstalling aiohttp-3.13.1:
      Successfully uninstalled aiohttp-3.13.1


In [ ]:
import os
import asyncio
import tempfile
import json
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
from aiogram.types import FSInputFile


# ────────────────────────────────


bot = Bot(token=TELEGRAM_TOKEN)
dp = Dispatcher()

# ────────────────────────────────
@dp.message(Command("start"))
async def start(message: types.Message):
    await message.answer(
        "👋 Привет! Отправь мне PDF, JPG или PNG документ, "
        "и я извлеку из него JSON с реквизитами и номенклатурой."
    )

@dp.message(lambda m: m.document or m.photo)
async def handle_file(message: types.Message):
    await message.answer("📄 Получил файл, начинаю обработку...")

    tmpdir = tempfile.mkdtemp()
    file_path = None

    # Сохраняем файл
    try:
        if message.document:
            file_info = await bot.get_file(message.document.file_id)
            file_path = os.path.join(tmpdir, message.document.file_name)
            await bot.download_file(file_info.file_path, file_path)
        elif message.photo:
            file_info = await bot.get_file(message.photo[-1].file_id)
            file_path = os.path.join(tmpdir, "image.jpg")
            await bot.download_file(file_info.file_path, file_path)
        else:
            await message.answer("❌ Поддерживаются только PDF, JPG и PNG.")
            return

        # Асинхронно вызываем твой обработчик
        loop = asyncio.get_event_loop()
        result = await loop.run_in_executor(None, extract_from_file, file_path)

        json_text = json.dumps(result, ensure_ascii=False, indent=2)

        if len(json_text) < 3900:
            await message.answer(
                f"✅ Результат:\n```json\n{json_text}\n```",
                parse_mode="Markdown"
            )
        else:
            json_file = os.path.join(tmpdir, "result.json")
            with open(json_file, "w", encoding="utf-8") as f:
                f.write(json_text)
            await message.answer_document(FSInputFile(json_file), caption="✅ Результат обработки")

    except Exception as e:
        await message.answer(f"⚠️ Ошибка при обработке файла: {e}")

    finally:
        # Очистка временных файлов
        try:
            if file_path and os.path.exists(file_path):
                os.remove(file_path)
        except Exception:
            pass

# ────────────────────────────────
if __name__ == "__main__":
    print("🤖 Бот запущен. Ожидаю сообщения...")

    import nest_asyncio
    nest_asyncio.apply()

    loop = asyncio.get_event_loop()
    loop.create_task(dp.start_polling(bot))
    loop.run_forever()

Exception ignored in: <coroutine object Dispatcher.start_polling at 0x7f2329b31cf0>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'
Exception ignored in: <coroutine object Dispatcher.start_polling at 0x7f2329b31cf0>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'


🤖 Бот запущен. Ожидаю сообщения...
JSON сохранён в /tmp/tmprewtfyu5/PDF000_extracted.json
